# LEET CODE
## Problem-1341: Movie Rating

Class: Medium

Source: https://leetcode.com/problems/movie-rating

## Description

Table: Movies
| Column Name   | Type    |
|---------------|---------|
| movie_id      | int     |
| title         | varchar |

- movie_id is the primary key (column with unique values) for this table.
- title is the name of the movie.

Table: Users
| Column Name   | Type    |
|---------------|---------|
| user_id       | int     |
| name          | varchar |
- user_id is the primary key (column with unique values) for this table.

Table: MovieRating
| Column Name   | Type    |
|---------------|---------|
| movie_id      | int     |
| user_id       | int     |
| rating        | int     |
| created_at    | date    |

- (movie_id, user_id) is the primary key (column with unique values) for this table.
- This table contains the rating of a movie by a user in their review.
- created_at is the user's review date. 

## To Do
Write a solution to:

- Find the name of the user who has rated the greatest number of movies. In case of a tie, return the lexicographically smaller user name.
- Find the movie name with the highest average rating in February 2020. In case of a tie, return the lexicographically smaller movie name.

The result format is in the following example.

## Example

Input: 

Movies table:
| movie_id    |  title       |
|-------------|--------------|
| 1           | Avengers     |
| 2           | Frozen 2     |
| 3           | Joker        |

Users table:
| user_id     |  name        |
|-------------|--------------|
| 1           | Daniel       |
| 2           | Monica       |
| 3           | Maria        |
| 4           | James        |

MovieRating table:
| movie_id    | user_id      | rating       | created_at  |
|-------------|--------------|--------------|-------------|
| 1           | 1            | 3            | 2020-01-12  |
| 1           | 2            | 4            | 2020-02-11  |
| 1           | 3            | 2            | 2020-02-12  |
| 1           | 4            | 1            | 2020-01-01  |
| 2           | 1            | 5            | 2020-02-17  | 
| 2           | 2            | 2            | 2020-02-01  | 
| 2           | 3            | 2            | 2020-03-01  |
| 3           | 1            | 3            | 2020-02-22  | 
| 3           | 2            | 4            | 2020-02-25  | 

Output: 
| results      |
|--------------|
| Daniel       |
| Frozen 2     |

Explanation: 
- Daniel and Monica have rated 3 movies ("Avengers", "Frozen 2" and "Joker") but Daniel is smaller lexicographically.
- Frozen 2 and Joker have a rating average of 3.5 in February but Frozen 2 is smaller lexicographically.


## Importing

In [1]:
import pandas as pd
from pandasql import sqldf

## Define Schema

In [3]:
data = {
    'movie_id': [1, 2, 3],
    'title': ['Avengers', 'Frozen 2', 'Joker']
}
Movies = pd.DataFrame(data)

# Create Users DataFrame
data = {
    'user_id': [1, 2, 3, 4],
    'name': ['Daniel', 'Monica', 'Maria', 'James']
}
Users = pd.DataFrame(data)

# Create MovieRating DataFrame
data = {
    'movie_id': [1, 1, 1, 1, 2, 2, 2, 3, 3],
    'user_id': [1, 2, 3, 4, 1, 2, 3, 1, 2],
    'rating': [3, 4, 2, 1, 5, 2, 2, 3, 4],
    'created_at': ['2020-01-12', '2020-02-11', '2020-02-12', '2020-01-01', '2020-02-17', '2020-02-01', '2020-03-01', '2020-02-22', '2020-02-25']
}
MovieRating = pd.DataFrame(data)

# Convert 'created_at' column to datetime type
MovieRating['created_at'] = pd.to_datetime(MovieRating['created_at'])

display(Movies)
display(Users)
display(MovieRating)

,movie_id,title
0,1,Avengers
1,2,Frozen 2
2,3,Joker


,user_id,name
0,1,Daniel
1,2,Monica
2,3,Maria
3,4,James


,movie_id,user_id,rating,created_at
0,1,1,3,2020-01-12
1,1,2,4,2020-02-11
2,1,3,2,2020-02-12
3,1,4,1,2020-01-01
4,2,1,5,2020-02-17
5,2,2,2,2020-02-01
6,2,3,2,2020-03-01
7,3,1,3,2020-02-22
8,3,2,4,2020-02-25


## Task

In [31]:
# Define the SQL query
query = """
WITH rated_user AS (
    SELECT user_id
    FROM MovieRating
    GROUP BY user_id
    HAVING COUNT(movie_id) = (SELECT MAX(cnt) FROM (SELECT COUNT(movie_id) AS cnt FROM MovieRating GROUP BY user_id))
),
rated_user_name AS (
    SELECT name AS results
    FROM rated_user AS ru
    LEFT JOIN Users USING(user_id)
    ORDER BY name ASC
    LIMIT 1
),
AvgRating AS (
    SELECT movie_id, AVG(rating) AS avg_rating
    FROM MovieRating
    WHERE strftime('%Y-%m', created_at) = '2020-02'
    GROUP BY movie_id
),
rated_movie AS (
    SELECT movie_id
    FROM AvgRating
    WHERE avg_rating = (
        SELECT MAX(avg_rating)
        FROM AvgRating
    )
),
rated_movie_name AS (
    SELECT title AS results
    FROM rated_movie
    LEFT JOIN Movies USING (movie_id)
    ORDER BY title ASC
    LIMIT 1
)
SELECT *
FROM rated_user_name

UNION ALL

SELECT *
FROM rated_movie_name
;
"""

# Excute the query using pandasql
result = sqldf(query, env={'Movies':Movies, 'Users':Users, 'MovieRating':MovieRating})

# Display the result dataframe
display(result)

,results
0,Daniel
1,Frozen 2


In [34]:
# Define the SQL query
query = """
SELECT DISTINCT FIRST_VALUE(u.name) OVER(ORDER BY COUNT(r.movie_id) DESC, u.name ASC) As results
FROM Users AS u 
LEFT JOIN MovieRating AS r USING(user_id)
GROUP BY u.user_id

UNION ALL

SELECT DISTINCT FIRST_VALUE(m.title) OVER(ORDER BY AVG(r.rating) DESC, m.title ASC) AS results
FROM Movies AS m
LEFT JOIN MovieRating AS r USING(movie_id)
WHERE strftime('%Y-%m', created_at) = '2020-02'
GROUP BY m.movie_id
;
"""

# Excute the query using pandasql
result = sqldf(query, env={'Movies':Movies, 'Users':Users, 'MovieRating':MovieRating})

# Display the result dataframe
display(result)

,results
0,Daniel
1,Frozen 2


In [35]:
## Mysql Submssion, as there is no DATE_FORMAT in sqlite
"""
SELECT DISTINCT FIRST_VALUE(u.name) OVER(ORDER BY COUNT(r.movie_id) DESC, u.name ASC) As results
FROM Users AS u 
LEFT JOIN MovieRating AS r USING(user_id)
GROUP BY u.user_id

UNION ALL

SELECT DISTINCT FIRST_VALUE(m.title) OVER(ORDER BY AVG(r.rating) DESC, m.title ASC) AS results
FROM Movies AS m
LEFT JOIN MovieRating AS r USING(movie_id)
WHERE DATE_FORMAT(created_at, '%Y-%m') = '2020-02'
GROUP BY m.movie_id
"""

"\nSELECT DISTINCT FIRST_VALUE(u.name) OVER(ORDER BY COUNT(r.movie_id) DESC, u.name ASC) As results\nFROM Users AS u \nLEFT JOIN MovieRating AS r USING(user_id)\nGROUP BY u.user_id\n\nUNION ALL\n\nSELECT DISTINCT FIRST_VALUE(m.title) OVER(ORDER BY AVG(r.rating) DESC, m.title ASC) AS results\nFROM Movies AS m\nLEFT JOIN MovieRating AS r USING(movie_id)\nWHERE DATE_FORMAT(created_at, '%Y-%m') = '2020-02'\nGROUP BY m.movie_id\n"

In [36]:
"""
WITH rated_user AS (
    SELECT user_id
    FROM MovieRating
    GROUP BY user_id
    HAVING COUNT(movie_id) = (SELECT MAX(cnt) FROM (SELECT COUNT(movie_id) AS cnt FROM MovieRating GROUP BY user_id) AS cnt_movie)
),
rated_user_name AS (
    SELECT name AS results
    FROM rated_user AS ru
    LEFT JOIN Users USING(user_id)
    ORDER BY name ASC
    LIMIT 1
),
AvgRating AS (
    SELECT movie_id, AVG(rating) AS avg_rating
    FROM MovieRating
    WHERE DATE_FORMAT(created_at, '%Y-%m') = '2020-02'
    GROUP BY movie_id
),
rated_movie AS (
    SELECT movie_id
    FROM AvgRating
    WHERE avg_rating = (
        SELECT MAX(avg_rating)
        FROM AvgRating
    )
),
rated_movie_name AS (
    SELECT title AS results
    FROM rated_movie
    LEFT JOIN Movies USING (movie_id)
    ORDER BY title ASC
    LIMIT 1
)
SELECT *
FROM rated_user_name

UNION ALL

SELECT *
FROM rated_movie_name
;
"""

"\nWITH rated_user AS (\n    SELECT user_id\n    FROM MovieRating\n    GROUP BY user_id\n    HAVING COUNT(movie_id) = (SELECT MAX(cnt) FROM (SELECT COUNT(movie_id) AS cnt FROM MovieRating GROUP BY user_id) AS cnt_movie)\n),\nrated_user_name AS (\n    SELECT name AS results\n    FROM rated_user AS ru\n    LEFT JOIN Users USING(user_id)\n    ORDER BY name ASC\n    LIMIT 1\n),\nAvgRating AS (\n    SELECT movie_id, AVG(rating) AS avg_rating\n    FROM MovieRating\n    WHERE DATE_FORMAT(created_at, '%Y-%m') = '2020-02'\n    GROUP BY movie_id\n),\nrated_movie AS (\n    SELECT movie_id\n    FROM AvgRating\n    WHERE avg_rating = (\n        SELECT MAX(avg_rating)\n        FROM AvgRating\n    )\n),\nrated_movie_name AS (\n    SELECT title AS results\n    FROM rated_movie\n    LEFT JOIN Movies USING (movie_id)\n    ORDER BY title ASC\n    LIMIT 1\n)\nSELECT *\nFROM rated_user_name\n\nUNION ALL\n\nSELECT *\nFROM rated_movie_name\n;\n"